In [5]:
import os
import joblib
import numpy as np
import pandas as pd
import polars as pl

import pandas.api.types
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import lightgbm as lgb

import lifelines
from lifelines.utils import concordance_index

import sys

print(sys.version)
for i in [np, pd, pl, mpl, sns, lifelines, sklearn]:
    try:
        print(i.__name__, i.__version__)
    except:
        print(i.__name__)

3.12.3 (main, May  1 2024, 17:33:23) [GCC 11.4.0]
numpy 1.26.4
pandas 2.2.2
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
lifelines 0.30.0
sklearn 1.4.2


In [6]:
import dproc, sgutil, sgpp, sgml, custpp
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer

data_path = 'data'
img_path = 'img'
result_path = 'result'
model_path = 'model'

sc = sgutil.SGCache(img_path, result_path)

In [7]:
X_4 = [
    'psych_disturb', 'diabetes', 'arrhythmia', 'renal_issue', 'pulm_severe', 'obesity', 'hepatic_severe', 
    'prior_tumor', 'peptic_ulcer','rheum_issue', 'hepatic_mild', 'cardiac','pulm_moderate'
]

X_int_fm = [
    'comorbidity_score', 'hla_high_res_10', 'hla_high_res_6', 'hla_high_res_8', 'hla_low_res_10',
    'hla_low_res_6', 'hla_low_res_8', 'hla_match_a_high', 'hla_match_a_low', 'hla_match_b_high',
    'hla_match_b_low', 'hla_match_drb1_low', 'hla_match_c_high', 'hla_match_c_low', 'hla_match_dqb1_high', 'hla_match_dqb1_low',
    'hla_match_drb1_high', 'hla_nmdp_6', 'karnofsky_score'
]

X_2 = [
    ('graft_type', {'peripheral blood': 0, 'bone marrow': 1}),
    ('prod_type', {'pb': 0, 'bm': 1}),
    ('vent_hist', {'no': -1, 'yes': 1}),
    ('rituximab', {'no': -1, 'yes': 1}),
    ('mrd_hct', {'negative': -1, 'positive': 1}),
    ('in_vivo_tcd', {'no': -1, 'yes': 1}),
    ('melphalan_dose', {'n/a, mel not given': -1, 'mel': 1})
]
X_na = X_4 +  X_int_fm +  ['donor_age']
X_nom_na = ['cmv_status', 'conditioning_intensity', 'cyto_score', 'cyto_score_detail', 'donor_related',
    'dri_score', 'ethnicity', 'gvhd_proph', 'sex_match', 'tce_div_match', 'tce_imm_match', 'tce_match'
]
X_nom_nna = ['prim_disease_hct', 'race_group', 'tbi_status']
p1 = make_pipeline(
    sgpp.PolarsProcessor(),
    sgpp.PandasCoverter(index_col = 'ID'),
    custpp.CIBMTTransformer(X_2, X_4, X_na, X_nom_na + X_nom_nna),
    sgpp.ApplyWrapper(SimpleImputer(strategy='most_frequent').set_output(transform='pandas'), X_int_fm, postfix = '_fm'),
    sgpp.ApplyWrapper(SimpleImputer(strategy='mean').set_output(transform='pandas'), ['donor_age'], postfix = '_fm'),
    sgpp.ApplyWrapper(sgpp.CatArrangerFreq(1, 'na', 'na'), X_nom_na),
    sgpp.ApplyWrapper(sgpp.CatOOVFilter(), X_nom_nna),
)
df_train = p1.fit_transform([os.path.join(data_path, 'train.csv')])

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
skf = StratifiedKFold(5, random_state = 123, shuffle = True)
for train_idx, valid_idx in skf.split(df_train[X_all], df_train['efs']):
    df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    t1, t2 = (
        df_cv_train.loc[df_cv_train['efs'] == 0.0, 'efs_time'].quantile(0.025),
        df_cv_train.loc[df_cv_train['efs'] == 1.0, 'efs_time'].quantile(0.6)
    )
    df_cv_train = df_cv_train.assign(efs_b = np.nan)
    df_cv_train.loc[(df_cv_train['efs'] == 0.0) & (df_cv_train['efs_time'] > t1), 'efs_b'] = 0
    df_cv_train.loc[(df_cv_train['efs'] == 1.0) & (df_train['efs_time'] < t2), 'efs_b'] = 1
    df_train_b = df_cv_train.loc[df_cv_train['efs_b'].notna()]
    print(t1, t2, len(df_train_b), len(df_train) - len(df_train_b))
    clf_lgb = lgb.LGBMClassifier(verbose = -1, n_estimators = 500, learning_rate = 0.02, class_weight={0.0: 1.0, 1.0: 1.0})
    clf_lgb.fit(df_train_b[X_all], df_train_b['efs_b'], categorical_feature = X_nom)
    prd1 = clf_lgb.predict_proba(df_train_b[X_all])[:, 1]
    prd2 = clf_lgb.predict_proba(df_cv_train[X_all])[:, 1]
    prd3 = clf_lgb.predict_proba(df_valid[X_all])[:, 1]
    print(
        roc_auc_score(df_train_b['efs'], prd1),
        roc_auc_score(df_cv_train['efs'], prd2),
        roc_auc_score(df_valid['efs'], prd3),
        concordance_index(df_train_b['efs_time'], -pd.Series(prd1, index = df_train_b.index), df_train_b['efs']),
        concordance_index(df_cv_train['efs_time'], -pd.Series(prd2, index = df_cv_train.index), df_cv_train['efs']),
        concordance_index(df_valid['efs_time'], -pd.Series(prd3, index = df_valid.index), df_valid['efs'])
    )